In [1]:
import theano
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Merge
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l1

import numpy as np
import pickle
import nltk
import re
import argparse

%matplotlib inline 
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


Using Theano backend.


//anaconda/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
from sklearn.metrics import mean_squared_error
def performance(y, y_pred):
     return mean_squared_error(y, y_pred)**0.5

In [3]:
import pandas as pd

working_dir = '/Users/praveen/work/data/kaggle/homeDepotRelPredict/'

# Load Training Data
train_data = pd.read_csv(working_dir + 'train.csv')
product_data = pd.read_csv(working_dir + 'product_descriptions.csv')
attributes_data = pd.read_csv(working_dir + 'attributes.csv')
qTxt = list(train_data['search_term'])
titleTxt = list(train_data['product_title'])

In [4]:
# Tokenize QueryText and Title Text
tokenizer = Tokenizer(50000)
tokenizer.fit_on_texts(titleTxt)

In [5]:
X_title = tokenizer.texts_to_matrix(titleTxt, mode='tfidf')
X_query = tokenizer.texts_to_matrix(qTxt, mode='tfidf')

In [6]:
#X_title = sequence.pad_sequences(titleSequences, maxlen=100)
#X_query = sequence.pad_sequences(querySequences, maxlen=20)
Y_all = train_data['relevance'] 

In [9]:
epochs = 5
print "building graph..."
from keras.regularizers import l2, activity_l2
query_model = Sequential()
#query_model.add(Embedding(10000, 128, input_length=10))
#query_model.add(GRU(128))
#query_model.add(Dropout(0.5))
query_model.add(Dense(16, input_dim=50000, activation='sigmoid'))
query_model.add(Dropout(0.5))




text_model = Sequential()
#text_model.add(Embedding(10000, 128, input_length=100))
#text_model.add(GRU(128))
#text_model.add(Dropout(0.5))
text_model.add(Dense(16,input_dim=50000, activation='sigmoid'))
text_model.add(Dropout(0.5))

combined_model = Sequential()
combined_model.add(Merge([query_model, text_model], mode='dot'))
combined_model.add(Dense(64))
combined_model.add(Dropout(0.5))
combined_model.add(Dense(1))
combined_model.compile(loss='mse', optimizer='adam')
print "done."


building graph...
done.


In [10]:
batch_size = 500
epoch = 5
startIndex = 0
endIndex = int(len(X_query) * .8) 
combined_model.fit([X_query[startIndex:endIndex], X_title[startIndex:endIndex]], Y_all[startIndex:endIndex], batch_size=batch_size, nb_epoch=epoch, validation_split=.2,shuffle=True, show_accuracy=False)

Train on 47402 samples, validate on 11851 samples
Epoch 1/5
47402/47402 [==============================] - 114s - loss: 2.2325 - val_loss: 0.4161
Epoch 2/5
47402/47402 [==============================] - 116s - loss: 1.2098 - val_loss: 0.3693
Epoch 3/5
47402/47402 [==============================] - 119s - loss: 0.8002 - val_loss: 0.3201
Epoch 4/5
47402/47402 [==============================] - 120s - loss: 0.5721 - val_loss: 0.2927
Epoch 5/5
47402/47402 [==============================] - 125s - loss: 0.4620 - val_loss: 0.2857


In [ ]:
startIndex = int(len(X_query) * .8)
endIndex = len(X_query)
#startIndex =int(len(X_query) * .3)
#endIndex = int(len(X_query) * .9) 
labels = Y_all[startIndex:endIndex]
print "predicting..."
y_pred = combined_model.predict_proba([X_query[startIndex:endIndex], X_title[startIndex:endIndex]], batch_size=batch_size)
performance(y_pred, labels)


In [ ]:
plt.hist(y_pred)
plt.show()

In [ ]:
startIndex = 0
endIndex = int(len(X_query) * .8)
#startIndex = int(len(X_query) * .8)
#endIndex = len(X_query)
labels = Y_all[startIndex:endIndex]
plt.hist(list(labels))
plt.show()

In [ ]:
y_pred = combined_model.evaluate([X_query[startIndex:endIndex], X_title[startIndex:endIndex]],Y_all[startIndex:endIndex], batch_size=batch_size, verbose=1)
print y_pred**0.5